In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import random_split


In [ ]:
# This is standard CIFAR image processing, including normalization

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# Let's set batch size to 4
batch_size = 4

# Load the training and testing sets from the CIFAR dataset

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)


trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)


testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane','car')

In [ ]:
# Subset the training and testing set to just planes and cars

def cycle(iterable):
    while True:
        for x in iterable:
            yield x

# Return only images of certain class (eg. aeroplanes = class 0)
def get_same_index(target, label1, label2):
    label_indices = []
    for i in range(len(target)):
        if target[i] == label1 or target[i] == label2:
            label_indices.append(i)
    return label_indices


# Get indices of label_class
train_indices = get_same_index(trainset.targets, 0, 1)

new_set = torch.utils.data.Subset(trainset, train_indices)

val_size = 4000
train_size = len(new_set) - val_size

train_ds, val_ds = random_split(new_set, [train_size, val_size])
len(train_ds), len(val_ds)

trainloader = torch.utils.data.DataLoader(dataset=train_ds, shuffle=True,
                                           batch_size=batch_size, drop_last=True)

validloader = torch.utils.data.DataLoader(dataset=val_ds, shuffle=True,
                                           batch_size=batch_size, drop_last=True)

# Get indices of label_class
test_indices = get_same_index(testset.targets, 0, 1)

new_set = torch.utils.data.Subset(testset, test_indices)

testloader = torch.utils.data.DataLoader(dataset=new_set, shuffle=True,
                                           batch_size=batch_size, drop_last=True)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# functions to show an image


def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()


# get some random training images
dataiter = iter(trainloader)
images, labels = dataiter.next()

# show images
imshow(torchvision.utils.make_grid(images))
print(' '.join('%5s' % classes[labels[j]] for j in range(batch_size)))

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

# this is our Neural Net

# change X to determine the number of neurons in the fully connected layer
X = 2

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        """YOUR CODE HERE"""
    
        # 3 input image channel(r,g,b), 50 output channels (=# of filters), 5x5 square convolution, stride = 1, padding = 'same'
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=50, kernel_size=5, stride=1, padding='same')
        self.conv2 = nn.Conv2d(in_channels=50, out_channels=50, kernel_size=5, stride=1, padding='same')
        self.pool1 = nn.MaxPool2d(kernel_size = 2, stride = 2)

        self.conv3 = nn.Conv2d(in_channels=50, out_channels=50, kernel_size=3, stride=1, padding='same')
        self.conv4 = nn.Conv2d(in_channels=50, out_channels=50, kernel_size=3, stride=1, padding='same')
        self.pool2 = nn.MaxPool2d(kernel_size = 4, stride = 4)

        self.linear1 = nn.Linear(800, X)
        
        self.linear2 = nn.Linear(X, 2)

        self.batchnorm = nn.BatchNorm2d(50)
        self.dropout = nn.Dropout(p=0.1)

        # The CrossEntropyLoss computes the softmax of the scores first, so you do
        # not need to add a separate softmax layer.
        self.loss = torch.nn.CrossEntropyLoss(reduction="mean")

    def forward(self, X):
        """
        Computes the forward-pass of the network.

        Parameters:
          X (tensor): input tensor of size (B, 3, 32, 32) where B is the batch size
        Returns:
          out (tensor): output scores of size (B, 10)
        """
        #conv layer 1
        data = self.dropout(F.relu(self.batchnorm(self.conv1(X))))

        #conv layer 2
        data = self.dropout(F.relu(self.batchnorm(self.conv2(data))))

        #maxpool layer
        data = self.pool1(data)

        #conv layer 3
        data = self.dropout(F.relu(self.batchnorm(self.conv3(data))))

        #conv layer 4
        data = self.dropout(F.relu(self.batchnorm(self.conv4(data))))

        #maxpool layer 2
        data = self.pool2(data)

        #flatten and linear (this is the fully connected layer)
        data = torch.flatten(data, start_dim=1)
        data = F.relu(self.linear1(data))
        data = self.linear2(data)

        return data


net = Net()

In [ ]:
import torch.optim as optim

# set optimizer parameters

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [ ]:
# we run this training for 5 epochs

for epoch in range(5):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 1000 == 999:    # print every 1000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 1000))
            running_loss = 0.0

print('Finished Training')

In [ ]:
# save the neurnal net
PATH = './new.pth'
torch.save(net.state_dict(), PATH)

In [ ]:
dataiter = iter(testloader)
images, labels = dataiter.next()

# print images
imshow(torchvision.utils.make_grid(images))
print('GroundTruth: ', ' '.join('%5s' % classes[labels[j]] for j in range(4)))

In [ ]:

net = Net()
net.load_state_dict(torch.load(PATH))

In [ ]:
outputs = net(images)

In [ ]:
_, predicted = torch.max(outputs, 1)

print('Predicted: ', ' '.join('%5s' % classes[predicted[j]]
                              for j in range(4)))

In [ ]:
# Get test set accuracy

correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in testloader:
        images, labels = data
        # calculate outputs by running images through the network
        outputs = net(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 4000 test images: %d %%' % (
    100 * correct / total))

In [ ]:
# Get training set accuracy

correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in trainloader:
        images, labels = data
        # calculate outputs by running images through the network
        outputs = net(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 6000 train images: %d %%' % (
    100 * correct / total))

In [ ]:
# Get validation set accuracy

correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in validloader:
        images, labels = data
        # calculate outputs by running images through the network
        outputs = net(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 4000 validation images: %d %%' % (
    100 * correct / total))
print(correct)

In [ ]:
# Let's get the output of the convolutional layers; we'll need this to determine
# what the MEC of the data set is
# We eliminate the FC layers from the neural net
newmodel = torch.nn.Sequential(*(list(net.children())[:-5]))

In [ ]:
# Iterate through the training and validation set and produce the output of the convolutional
# layers, create a list of np arrays of size 1x800, which contain all of the features
# for each training set image

total = 0
output = []
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in trainloader:
        images, labels = data
        # calculate outputs by running images through the network
        outputs = newmodel(images)
        total+=1
        for i in range(4):
            output.append(np.append(torch.flatten(outputs, start_dim=1)[i].numpy(),labels[i]))
    for data in validloader:
        images, labels = data
        # calculate outputs by running images through the network
        outputs = newmodel(images)
        total+=1
        for i in range(4):
            output.append(np.append(torch.flatten(outputs, start_dim=1)[i].numpy(),labels[i]))


In [ ]:
# Add the header column to the list of outputs

output.insert(0,np.append(np.arange(800),"Label"))

In [ ]:
# Now we write this a csv that we can directly input into 
# our nntailoring function and determine the MEC

import csv

with open('output_5_5.csv', 'w') as f:
    csvwriter = csv.writer(f)
    csvwriter.writerows(output)